In [1]:
import sys
sys.path.append('..')
from src.data.make_dataset import data_preparation
from src.features.build_features import cbk_feature_engineering

In [2]:
import pandas as pd
import joblib
import numpy as np

# 1. Load and preprocess data

In [3]:
unlabeled_data = pd.read_excel('../data/raw/Missão_Stone_-_Dados_de_trx_(3).xlsx', engine='openpyxl', sheet_name='Aba 2')

In [4]:
def cbk_data_prep(df, test=False):
    df = data_preparation(df, test)
    df = cbk_feature_engineering(df)
    return df

processed_test = cbk_data_prep(unlabeled_data, test=True)

In [5]:
for col in ['periodo_do_dia', 'dia_da_semana', 'emissor_cartao']:
    processed_test[col] = processed_test[col].astype('category')

## 1.1 Model results

In [6]:
results = pd.read_csv(f'../models/results_model_2024-06-10_13h34m.csv')
results

,Accuracy,Precision,Recall,F1-Score,AUC-ROC,KS,opt_prob_cutoff
0,86.14,23.4,83.81,36.59,85.03,70.07,0.031236


# 2. Predict

In [7]:
results['opt_prob_cutoff'][0]

0.0312358652774534

In [8]:
model = joblib.load('../models/model_2024-06-10_13h34m.pkl')
y_pred_prob = model.predict_proba(processed_test.drop(columns='CBK'))[:,1]
y_pred_class = (y_pred_prob >= results['opt_prob_cutoff'][0]).astype("int")
y_pred = pd.DataFrame(y_pred_class, columns=['predicted_CBK'])
y_pred_proba = pd.DataFrame(y_pred_prob, columns=['predicted_CBK_proba'])

In [9]:
predicted_data = pd.concat([processed_test.drop(columns='CBK'), y_pred, y_pred_proba], axis=1).reset_index(drop=True)
predicted_data.head(3)

,Valor,periodo_do_dia,dia_da_semana,emissor_cartao,predicted_CBK,predicted_CBK_proba
0,112.00,madrugada,Segunda-feira,41555,0,0.015386
1,112.00,madrugada,Segunda-feira,06669,0,0.001894
2,18.34,madrugada,Segunda-feira,41187,0,0.006643


In [10]:
predicted_data.shape

(11820, 6)

In [11]:
predicted_data.predicted_CBK.value_counts()

predicted_CBK
0    10046
1     1774
Name: count, dtype: int64

In [12]:
predicted_data.to_csv('../data/predicted/aba2_predicted.csv', index=False)

# 3. Avaliação de negócio

## 3.1 Proposta de tratativa

Se um modelo classifica e ordena bem as probabilidades, poderiamos criar um score de 'propensão' a chargeback para criar regras de negócio e adições a um sistema de avaliação near real-time que cancele e envie para análise junto com algum outro modelo a transação com alto risco de ser um chargeback fraudulento.

É claro, para esse desafio temos poucos dados para realmente afirmar que é um chargeback fraudulento ou se houve erro próprio, então precisaria de mais dados para não haver perdas em compras, como por exemplo, dados cadastrais e dados informados em compra para comparação, validação de cpf, dados de acesso entre outros.

## 3.2 Análise de valor do modelo

Assim, vamos observar o cenário do modelo atual.

| classe | precision | recall | f1-score | support
|--------|-----------|--------|----------|----------
| 0      | 0.99      | 0.86   | 0.92     | 2096
| 1      | 0.23      | 0.84   | 0.37     | 105

Ou seja:\
% das transações chargeback nesta parcela = 105/(2096+105) = 4,77%\
A acurácia do modelo geral é 86%

Nos dados do outro mês temos 11820 casos. De acordo com as probabilidades, temos, em tese:\
~564 casos de chargeback (4,77% de 11820)\
~11256 casos de não chargeback (demais)

Dos casos de chargeback, devido a acurácia:\
485 seriam chargeback e o modelo teria dito que é chargeback (certo)\
79 seriam chargeback e o modelo teria dito que não é chargeback (perda)

Dos casos que não são chargeback\
9.680 não seriam chargeback e o modelo teria dito que não é chargeback (certo)\
1.576 não seriam chargeback e o modelo teria dito que é chargeback (perda)

Então, usando essa estatística para tentar prever economia no dataset de predict:


In [13]:
# Ordenando pela 'confiança'
predicted_data_cbk = predicted_data[predicted_data['predicted_CBK']==1].copy().sort_values(by='predicted_CBK_proba', ascending=False)
predicted_data_not_cbk = predicted_data[~(predicted_data['predicted_CBK']==1)].copy().sort_values(by='predicted_CBK_proba', ascending=True)

In [14]:
print(f'CBK data points:{predicted_data_cbk.shape[0]}')
print(f'Não CBK data points:{predicted_data_not_cbk.shape[0]}')

CBK data points:1774
Não CBK data points:10046


In [15]:
# Supostamente, dos datapoints cbk ele teria acertado 86% -> 1774 * 0,86 = 1.206
cbk_certo = predicted_data_cbk.head(1208) # economia, pegando os valores que o modelo tem mais certeza
cbk_errado = predicted_data_cbk.tail(568) # perda

In [16]:
print(f"Economia em classificados como chargeback: {round(np.sum(cbk_certo['Valor'])) - round(np.sum(cbk_errado['Valor']))}")

Economia em classificados como chargeback: 135150


#### 3.2.1 Uma outra visão desta estatística em faixas:

In [17]:
predicted_data_score = predicted_data.copy()

In [18]:
prob_mean = np.mean(predicted_data_score['predicted_CBK_proba'])
prob_std = np.std(predicted_data_score['predicted_CBK_proba'])
predicted_data_score['Z_score_lgbm'] = (predicted_data_score['predicted_CBK_proba']-prob_mean)/prob_std
predicted_data_score['score_lgbm'] = round(1000*(predicted_data_score['Z_score_lgbm']-min(predicted_data_score['Z_score_lgbm']))/(max(predicted_data_score['Z_score_lgbm'])-min(predicted_data_score['Z_score_lgbm'])),0)

predicted_data_score['FAIXAS_CBKSCORE_LGBM'] = pd.qcut(predicted_data_score['score_lgbm'],
                                                q=20,duplicates='drop')


In [19]:
cbkscore=pd.pivot_table(data=predicted_data_score, index='FAIXAS_CBKSCORE_LGBM',
               values='predicted_CBK',
               aggfunc={'count',sum},
               fill_value=0)
cbkscore['cbk_rate'] = round(100*cbkscore['sum']/cbkscore['count'],2)
cbkscore['cbk_pct'] = round(100*cbkscore['sum'].cumsum()/np.sum(cbkscore['sum']),2)

cbkscore = cbkscore.sort_values(by='cbk_rate')

cbkscore['cbk_rate'] = round(100*cbkscore['sum']/cbkscore['count'],2)
cbkscore['cbk_pct'] = round(100*cbkscore['sum'].cumsum()/np.sum(cbkscore['sum']),2)

cbkscore

C:\Users\letic\AppData\Local\Temp\ipykernel_461564\3962318178.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  cbkscore=pd.pivot_table(data=predicted_data_score, index='FAIXAS_CBKSCORE_LGBM',
C:\Users\letic\AppData\Local\Temp\ipykernel_461564\3962318178.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  cbkscore=pd.pivot_table(data=predicted_data_score, index='FAIXAS_CBKSCORE_LGBM',


,count,sum,cbk_rate,cbk_pct
FAIXAS_CBKSCORE_LGBM,,,,
"(-0.001, 1.0]",3317,0,0.00,0.00
"(1.0, 2.0]",1055,0,0.00,0.00
"(2.0, 3.0]",846,0,0.00,0.00
"(3.0, 4.0]",678,0,0.00,0.00
"(4.0, 5.0]",586,0,0.00,0.00
"(5.0, 6.0]",453,0,0.00,0.00
"(6.0, 7.0]",367,0,0.00,0.00
"(7.0, 9.0]",582,0,0.00,0.00
"(9.0, 11.0]",436,0,0.00,0.00
